In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("stanfordnlp/imdb")
train_dataset = dataset["train"]
#val_dataset = dataset["validation"]
test_dataset = dataset["test"]

Using the latest cached version of the dataset since stanfordnlp/imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/dhruv/.cache/huggingface/datasets/stanfordnlp___imdb/plain_text/0.0.0/e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Sat Apr 20 22:44:49 2024).


In [3]:
for item in test_dataset:
    if item['label'] == 0:
        print(item['text'])
        print()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Use BERT to have single vector for sentence

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/dhruv/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# Load pre-trained BERT model and tokenizer
# model_name = "bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

In [6]:
# def get_doc_embedding(document, tokenizer, model):
#     # Tokenize input text
#     inputs = tokenizer(document, return_tensors="pt", padding=True, truncation=True)
#     # Forward pass through the model
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Extract document embedding from hf model output
#     document_embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0)
#     return document_embedding.reshape(-1, 1)

def get_doc_embedding(document, model):
    return model.encode(document)

In [7]:
# Example document from GLUE SST dataset
document = "those so-so films that could have been much better"
doc_emb = get_doc_embedding(document, model)
print(doc_emb.shape)


(768,)


### Save the vectors in pkl files

In [8]:
import tqdm
import numpy as np
import pickle

In [9]:
doc_vectors = [] 
# Init dummylist to decode whihc split train/val/test
dummy_list = test_dataset

In [10]:
for item in tqdm.tqdm(dummy_list):
    sentence = item['text']
    doc_emb = get_doc_embedding(sentence, model)
    doc_vectors.append(doc_emb)

100%|██████████| 25000/25000 [1:48:33<00:00,  3.84it/s]  


In [11]:
# Open the file and store the reduced sentence vectors
with open('imdb-test-768.pkl', 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(doc_vectors, file)

### Use PCA to reduce vector dimensions

In [37]:
# doc_vectors_np = np.array(doc_vectors)

# # 768 is emb dimension
# doc_vec_rehsaped = doc_vectors_np.reshape((len(dummy_list),768))

# # Apply PCA for dimensionality reduction
# pca = PCA(n_components = target_dimension)
# pca.fit(doc_vec_rehsaped)

# final_vecs = pca.transform(doc_vec_rehsaped)